In [58]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

In [59]:
# Fetch dataset 
bank_marketing = fetch_ucirepo(id = 222) 
  
# Data (as pandas dataframes) 
X = bank_marketing.data.features
y = bank_marketing.data.targets

print(f"Features shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")

Features shape: (45211, 16)
Target distribution:
y  
no     39922
yes     5289
Name: count, dtype: int64


In [60]:
data = bank_marketing.data.original.copy()
data = data.convert_dtypes()
# Move last column to the first position
data = data[[data.columns[-1]] + list(data.columns[:-1])]
# Probably the name of the column 'day_of_week' is a mistake
# changing it to 'day_of_month' (Look at the data description)
data.rename(columns = {'day_of_week': 'day_of_month'}, inplace = True)
data.to_excel('../Data/bank_marketing_data.xlsx', index = False)
data.head()

,y,age,job,marital,education,default,balance,housing,loan,contact,day_of_month,month,duration,campaign,pdays,previous,poutcome
0,no,58,management,married,tertiary,no,2143,yes,no,<NA>,5,may,261,1,-1,0,<NA>
1,no,44,technician,single,secondary,no,29,yes,no,<NA>,5,may,151,1,-1,0,<NA>
2,no,33,entrepreneur,married,secondary,no,2,yes,yes,<NA>,5,may,76,1,-1,0,<NA>
3,no,47,blue-collar,married,<NA>,no,1506,yes,no,<NA>,5,may,92,1,-1,0,<NA>
4,no,33,<NA>,single,<NA>,no,1,no,no,<NA>,5,may,198,1,-1,0,<NA>


In [61]:
# Get a concise summary of the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   y             45211 non-null  string
 1   age           45211 non-null  Int64 
 2   job           44923 non-null  string
 3   marital       45211 non-null  string
 4   education     43354 non-null  string
 5   default       45211 non-null  string
 6   balance       45211 non-null  Int64 
 7   housing       45211 non-null  string
 8   loan          45211 non-null  string
 9   contact       32191 non-null  string
 10  day_of_month  45211 non-null  Int64 
 11  month         45211 non-null  string
 12  duration      45211 non-null  Int64 
 13  campaign      45211 non-null  Int64 
 14  pdays         45211 non-null  Int64 
 15  previous      45211 non-null  Int64 
 16  poutcome      8252 non-null   string
dtypes: Int64(7), string(10)
memory usage: 6.2 MB


## Managing NAs for Categorical Variables

In [62]:
# replace NaN with a custom label
data['job'] = data['job'].fillna("Not Specified")
data['education'] = data['education'].fillna("Not Specified")
data['contact'] = data['contact'].fillna("Not Specified")
data['poutcome'] = data['poutcome'].fillna("Not Specified")
data.info() # Yay! No missing values in data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   y             45211 non-null  string
 1   age           45211 non-null  Int64 
 2   job           45211 non-null  string
 3   marital       45211 non-null  string
 4   education     45211 non-null  string
 5   default       45211 non-null  string
 6   balance       45211 non-null  Int64 
 7   housing       45211 non-null  string
 8   loan          45211 non-null  string
 9   contact       45211 non-null  string
 10  day_of_month  45211 non-null  Int64 
 11  month         45211 non-null  string
 12  duration      45211 non-null  Int64 
 13  campaign      45211 non-null  Int64 
 14  pdays         45211 non-null  Int64 
 15  previous      45211 non-null  Int64 
 16  poutcome      45211 non-null  string
dtypes: Int64(7), string(10)
memory usage: 6.2 MB


In [63]:
for col in data.select_dtypes(include = 'string').columns:
    print(f"Value counts for '{col}':")
    print(data[col].value_counts(dropna=False))
    print('-' * 40)

Value counts for 'y':
y
no     39922
yes     5289
Name: count, dtype: Int64
----------------------------------------
Value counts for 'job':
job
blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
Not Specified     288
Name: count, dtype: Int64
----------------------------------------
Value counts for 'marital':
marital
married     27214
single      12790
divorced     5207
Name: count, dtype: Int64
----------------------------------------
Value counts for 'education':
education
secondary        23202
tertiary         13301
primary           6851
Not Specified     1857
Name: count, dtype: Int64
----------------------------------------
Value counts for 'default':
default
no     44396
yes      815
Name: count, dtype: Int64
----------------------------------------
Value counts for 'housing':
housing
yes

In [64]:
data.describe()

,age,balance,day_of_month,duration,campaign,pdays,previous
count,45211.0,45211.0,45211.0,45211.0,45211.0,45211.0,45211.0
mean,40.93621,1362.272058,15.806419,258.16308,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.0,-8019.0,1.0,0.0,1.0,-1.0,0.0
25%,33.0,72.0,8.0,103.0,1.0,-1.0,0.0
50%,39.0,448.0,16.0,180.0,2.0,-1.0,0.0
75%,48.0,1428.0,21.0,319.0,3.0,-1.0,0.0
max,95.0,102127.0,31.0,4918.0,63.0,871.0,275.0


In [65]:
categorical_cols = ["job","marital","education","default","housing","loan","contact","month","poutcome"]
numeric_cols = ["age","balance","day_of_month","duration","campaign","pdays","previous"]
outcome_col = ["y"]

In [66]:
# create combo key (treat NaN as "__NA__"), pool ultra-rare combos into "__OTHER__"
combo = data[categorical_cols].fillna("__NA__").astype(str).agg("||".join, axis = 1)
vc = combo.value_counts()
combo = combo.where(~combo.isin(vc[vc < 2].index), "__OTHER__")

# choose stratification column: combo if valid else fallback to target y if present else None
strat = combo if (combo.value_counts() >= 2).all() else (data["y"] if "y" in data.columns else None)

# stratified 80/20 split (random_state for reproducibility)
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42, stratify = strat)

In [67]:
train_yes_pct = 100 * (train_data['y'].value_counts()['yes'] / len(train_data))
test_yes_pct = 100 * (test_data['y'].value_counts()['yes'] / len(test_data))

print(f'Train Test Ratio: {100*len(train_data)/len(data):.2f} : {100*len(test_data)/len(data):.2f}')
print(f"Train data 'yes' percentage: {train_yes_pct:.2f}%")
print(f"Test data 'yes' percentage: {test_yes_pct:.2f}%")

Train Test Ratio: 80.00 : 20.00
Train data 'yes' percentage: 11.70%
Test data 'yes' percentage: 11.69%


In [68]:
train_data.to_excel('../Data/train_data.xlsx', index = False)
test_data.to_excel('../Data/test_data.xlsx', index = False)